In [ ]:
import pandas as pd
import numpy as np

Notebook settings

In [29]:
# Automatically reloads any modules that are imported, 
# so that any changes made to the module files are reflected # without needing to restart the Jupyter kernel.
# load autoreload module
%load_ext autoreload
# mode 1 reloads only when an import statement is called. For production
# mode 2 reloads before execution of every cell
%autoreload 2

# limit the number of rows that are shown with printing dataframes
pd.set_option('display.max_rows', 5)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [30]:
df = pd.read_csv("data/cleaned/pubs.csv")

C:\Users\skirsch\AppData\Local\Temp\ipykernel_2028\887412386.py:1: DtypeWarning: Columns (16) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("data/cleaned/pubs.csv")


### Filter for biomedical institutions of recent years

In [ ]:
df_domain = df.copy()

# add domain column after institution columns
df_domain.insert(loc=df_domain.columns.get_loc("institution") + 1, column="domain", value=None)

df_domain

# biomedical institutions
biomedical = ['amcpub', 'lumc', 'vumc', 'umcu'] 

# pick a reference year for reproducibility
reference_year = 2024 # year where the study was conducted/started
year_diff = 5 # look up to 5 years back

date_n_years_ago = reference_year - year_diff

df_domain["domain"] = np.where(
    (df_domain["institution"].isin(biomedical)) & (df_domain["year"] >= date_n_years_ago),
    "biomedical_recent",
    None
)

df_domain

,integer_id,thesis_identifier,institution,domain,phd_name,title,year,language,contributor_1,contributor_2,contributor_3,contributor_4,contributor_5,contributor_6,contributor_7,contributor_8,contributor_9,contributor_10
0,1,https://pure.amc.nl/en/publications/structure-...,amcpub,None,"Azaripour, Adriano",Structure and function of the human periodonti...,2016,en,"van Noorden, Cornelis J. F.","Willershausen, B.",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,https://pure.amc.nl/en/publications/neurocogni...,amcpub,None,"Thissen, Andrieke J. A. M.",Neurocognitive and genetic factors in ADHD acr...,2014,en,"Buitelaar, J. K.","Franke, B.","Lambregts-Rommelse, N. N. J.",NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94297,125844,https://research.wur.nl/en/publications/variab...,wur,None,"van den Berg, J.A.",Variability of parameters for modelling soil m...,1989,en,"van der Molen, W.H.","Terwindt, J.H.J.",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
94298,125845,https://research.wur.nl/en/publications/overla...,wur,None,"de Lima, J.L.M.P.",Overland flow under rainfall : some aspects re...,1989,en,"van der Molen, W.H.","Koopmans, R.W.R.",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Filter out publications that are older than 5 years

### Additional cleaning

In [ ]:
df_filter = df_domain.copy()

# Remove all pipe characters from the phd_name column
df_filter['phd_name'] = df_filter['phd_name'].str.replace('|', '')

# get rid of contributors that start with comma or "Surgery"
contributor_columns = df_filter.filter(like="contributor_").columns # Apply the replacement only to the "contributor_" columns
df_filter[contributor_columns] = df_filter[contributor_columns].replace(
    {r'^,.*': np.nan, r'^Surgery.*': np.nan},
    regex=True
)

# Filter out rows where all "contributor_" columns are NaN
nan_rows = df_filter[contributor_columns].isnull().all(axis=1)
df_filter = df_filter[~nan_rows].reset_index(drop=True)

# Shift non-NaN values to the left in the contributor columns
# i.e. promote contributors until we have an uninterrupted sequence of contributors
df_filter[contributor_columns] = df_filter[contributor_columns].apply(
   lambda x: pd.Series(x.dropna().tolist() + [np.nan]*(len(x) - x.count())),
   axis=1 # apply function per row
)

df_filter


,integer_id,thesis_identifier,institution,domain,phd_name,title,year,language,contributor_1,contributor_2,contributor_3,contributor_4,contributor_5,contributor_6,contributor_7,contributor_8,contributor_9,contributor_10
0,1,https://pure.amc.nl/en/publications/structure-...,amcpub,None,"Azaripour, Adriano",Structure and function of the human periodonti...,2016,en,"van Noorden, Cornelis J. F.","Willershausen, B.",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,https://pure.amc.nl/en/publications/neurocogni...,amcpub,None,"Thissen, Andrieke J. A. M.",Neurocognitive and genetic factors in ADHD acr...,2014,en,"Buitelaar, J. K.","Franke, B.","Lambregts-Rommelse, N. N. J.",NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94072,125844,https://research.wur.nl/en/publications/variab...,wur,None,"van den Berg, J.A.",Variability of parameters for modelling soil m...,1989,en,"van der Molen, W.H.","Terwindt, J.H.J.",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
94073,125845,https://research.wur.nl/en/publications/overla...,wur,None,"de Lima, J.L.M.P.",Overland flow under rainfall : some aspects re...,1989,en,"van der Molen, W.H.","Koopmans, R.W.R.",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [33]:
df_final = df_filter.copy()
df_final = df_final.dropna(subset=['phd_name', 'title', 'contributor_1'])

df_final = df_final.reset_index(drop=True)

df_final

,integer_id,thesis_identifier,institution,domain,phd_name,title,year,language,contributor_1,contributor_2,contributor_3,contributor_4,contributor_5,contributor_6,contributor_7,contributor_8,contributor_9,contributor_10
0,1,https://pure.amc.nl/en/publications/structure-...,amcpub,None,"Azaripour, Adriano",Structure and function of the human periodonti...,2016,en,"van Noorden, Cornelis J. F.","Willershausen, B.",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,https://pure.amc.nl/en/publications/neurocogni...,amcpub,None,"Thissen, Andrieke J. A. M.",Neurocognitive and genetic factors in ADHD acr...,2014,en,"Buitelaar, J. K.","Franke, B.","Lambregts-Rommelse, N. N. J.",NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94072,125844,https://research.wur.nl/en/publications/variab...,wur,None,"van den Berg, J.A.",Variability of parameters for modelling soil m...,1989,en,"van der Molen, W.H.","Terwindt, J.H.J.",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
94073,125845,https://research.wur.nl/en/publications/overla...,wur,None,"de Lima, J.L.M.P.",Overland flow under rainfall : some aspects re...,1989,en,"van der Molen, W.H.","Koopmans, R.W.R.",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Save to CSV - USE IN `Open_Alex_Final.ipynb`

In [34]:
df_final.to_csv('data/cleaned/pubs_with_domain.csv', index=False)